In [48]:
import requests
import geopandas as gpd
import pandas as pd
import json
from shapely.geometry import shape
import numpy as np
from lonboard import Map, ScatterplotLayer

In [49]:
BASE_URL = "http://127.0.0.1:8000"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"

In [54]:
def fetch_admin_boundaries(iso3: str, adm: str) -> gpd.GeoDataFrame:
    """Fetch administrative boundaries from GeoBoundaries API."""
    url = f'https://www.geoboundaries.org/api/current/gbOpen/{iso3}/{adm}/'
    res = requests.get(url).json()
    return gpd.read_file(res['gjDownloadURL'])

def fetch_summary_data(feature: Dict) -> pd.DataFrame:
    """Fetch summary data for each administrative feature."""
    request_payload = {
        "aoi": feature,
        "spatial_join_method": "touches",
        "fields": ["sum_pop_2020"], 
        "geometry": "point"
    }
    response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
    if response.status_code != 200:
        raise Exception(f"Failed to get summary: {response.text}")
    
    summary_data = response.json()
    if not summary_data:
        print(f"Failed to get summary for {feature['id']}")
        return pd.DataFrame()  # Return an empty DataFrame if no data
    
    df = pd.DataFrame(summary_data)
    df['adm_id'] = int(feature['id'])
    df['adm_name'] = feature['properties']['shapeName']
    df['geometry'] = df['geometry'].apply(lambda geom: shape(geom))
    return df

In [55]:
ISO3 = 'KEN'
ADM = 'ADM1'
adm_boundaries = fetch_admin_boundaries(ISO3, ADM)
geojson_str = adm_boundaries.to_json()
adm_geojson = json.loads(geojson_str)
adm_features = adm_geojson['features']

gdfs = []
for i, feature in enumerate(adm_features):
    df = fetch_summary_data(feature)
    if not df.empty:
        gdfs.append(gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326'))
        print(feature['properties']['shapeName'], i)

# Concatenate all GeoDataFrames into a single GeoDataFrame
gdf = pd.concat(gdfs, ignore_index=True)

# Display the GeoDataFrame structure
gdf.head()

Turkana 0
Marsabit 1
Mandera 2
Wajir 3
West Pokot 4
Samburu 5
Isiolo 6
Baringo 7
Elgeyo-Marakwet 8
Trans Nzoia 9
Bungoma 10
Garissa 11
Uasin Gishu 12
Kakamega 13
Laikipia 14
Busia 15
Meru 16
Nandi 17
Siaya 18
Nakuru 19
Vihiga 20
Nyandarua 21
Tharaka 22
Kericho 23
Kisumu 24
Nyeri 25
Tana River 26
Kitui 27
Kirinyaga 28
Embu 29
Homa Bay 30
Bomet 31
Nyamira 32
Narok 33
Kisii 34
Murang'a 35
Migori 36
Kiambu 37
Machakos 38
Kajiado 39
Nairobi 40
Makueni 41
Lamu 42
Kilifi 43
Taita Taveta 44
Kwale 45
Mombasa 46


,hex_id,geometry,sum_pop_2020,adm_id,adm_name
0,866a4a407ffffff,POINT (35.68873 4.94892),554.655021,0,Turkana
1,866a4a417ffffff,POINT (35.59512 1.81273),207.977046,0,Turkana
2,866a4a41fffffff,POINT (35.17288 5.20383),637.218817,0,Turkana
3,866a4a427ffffff,POINT (35.61627 2.62098),948.876730,0,Turkana
4,866a4a437ffffff,POINT (35.89027 3.48441),112.805780,0,Turkana


In [56]:
# Define custom breaks and corresponding RGBA colors for visualization
breaks = [0, 1, 1000, 10000, 50000, 100000, 200000, gdf["sum_pop_2020"].max()]
colors = np.array([
    [211, 211, 211, 255],  # Light gray for 0
    [255, 255, 0, 255],    # Yellow for 1-1000
    [255, 165, 0, 255],    # Orange for 1000-10000
    [255, 0, 0, 255],      # Red for 10000-50000
    [128, 0, 128, 255],    # Purple for 50000-100000
    [0, 0, 255, 255],      # Blue for 100000-200000
    [0, 0, 139, 255],      # Dark blue for 200000+
])

def assign_color(value: float) -> list:
    """Assign colors based on population value."""
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i].tolist()  # Convert to list
    return colors[-1].tolist()  # In case value exceeds all breaks

In [58]:
# Map sum_pop_2020 values to colors using the custom function
gdf['color'] = gdf["sum_pop_2020"].apply(assign_color)

# Flatten the color list into a 2D array for ScatterplotLayer
color_list = np.array(gdf['color'].tolist(), dtype=np.uint8)

# Create the scatterplot layer with the assigned colors
layer = ScatterplotLayer.from_geopandas(gdf, get_radius=2000, get_fill_color=color_list)

# Create the map with the scatterplot layer
m = Map(layer)
m

Map(layers=[ScatterplotLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x13b3b78e0>
[
  [
    2…